In [2]:
import torch
import numpy as np

In [3]:
x = torch.empty(5, 3)

x = torch.rand(5, 3)

x = torch.zeros(5, 3, dtype=torch.long)

x = torch.tensor([5.5, 3])

#上述创建的都是一个tensor

In [4]:
#在已有的张量基础上。。。
x = x.new_ones(5, 3, dtype=torch.double) ##和x有相同属性：dtype，device，除非新指定

x = torch.randn_like(x, dtype=torch.float)#和x有相同size

# 加法

In [5]:
y = torch.rand(5, 3)
print(x + y)

tensor([[ 2.3762,  1.0624,  1.2193],
        [ 0.8053, -0.1782,  2.0064],
        [ 0.2975, -0.1549,  0.1682],
        [-0.1698,  0.1244, -1.5436],
        [-0.3300,  0.6852,  1.4709]])


In [6]:
#指定结果
result = torch.empty(5, 3)
torch.add(x, y, out=result)
print(result)

tensor([[ 2.3762,  1.0624,  1.2193],
        [ 0.8053, -0.1782,  2.0064],
        [ 0.2975, -0.1549,  0.1682],
        [-0.1698,  0.1244, -1.5436],
        [-0.3300,  0.6852,  1.4709]])


### `x.copy_(y), x.t_()` 这些带`_` 的都会改变x

In [7]:
#把x加到y上
y.add_(x)

tensor([[ 2.3762,  1.0624,  1.2193],
        [ 0.8053, -0.1782,  2.0064],
        [ 0.2975, -0.1549,  0.1682],
        [-0.1698,  0.1244, -1.5436],
        [-0.3300,  0.6852,  1.4709]])

# 操作

In [8]:
y.view(-1,5) #reshape

tensor([[ 2.3762,  1.0624,  1.2193,  0.8053, -0.1782],
        [ 2.0064,  0.2975, -0.1549,  0.1682, -0.1698],
        [ 0.1244, -1.5436, -0.3300,  0.6852,  1.4709]])

In [9]:
#numpy tensor相互转换。
a = np.ones(5)
b = torch.from_numpy(a)

a = torch.ones(5)
b = a.numpy()


In [10]:
#cuda tensor
if torch.cuda.is_available():
    device = torch.device("cuda")          # CUDA device object
    y = torch.ones_like(x, device=device)  # 直接创建cuda张量
    x = x.to(device)                       # 用 `.to()`
    z = x + y
    print(z)
    print(z.to("cpu", torch.double)) 

# Autograd

torch.Tensor的属性：

.requires_grad 是否需要计算梯度

.requires_grad_( ... ) 修改上述属性

.backward() 计算梯度 
**计算一次后,计算图被释放
retain_graph=True可以保持计算图**

.grad 梯度（需要先执行.backward()）
**一个张量的梯度会累计在.gard中!!!**

.detach() 停止追踪(会返回一个相同的张量,requires_grad=false)

with torch.no_grad(): 在这个代码块里的任何张量都不会被追踪(即使requires_grad=true)



In [14]:
x = torch.ones(2, 2, requires_grad=True)
y=x+2
z=y*y*2

#z是一个张量,无法直接求backward
# z.backward()

#需要:
# z.backward(torch.ones_like(x))
# print(x.grad)

out=z.mean()
out.backward()
print(x.grad)

tensor([[3., 3.],
        [3., 3.]])


# 深究backward() 的grad_tensors参数

In [15]:
x = torch.tensor([2, 3, 4], dtype=torch.float, requires_grad=True)
print(x)
y = x * 2
while y.norm() < 1000:
    y = y * 2
print(y)

y.backward(torch.ones_like(y))
print(x.grad)

tensor([2., 3., 4.], requires_grad=True)
tensor([ 512.,  768., 1024.], grad_fn=<MulBackward0>)
tensor([256., 256., 256.])


> 为什么在求导的过程中需要引入这个参数，如果我们不引入这个参数的话，则会报下面的错误：
>
> RuntimeError: grad can be implicitly created only for scalar outputs
>
> 即为提示我们输出不是一个标量

那实际上输出的是什么呢?

如果是(向量)矩阵对(向量)矩阵求导(tensor对tensor求导)

实际上是先求出Jacobian矩阵中每一个元素的梯度值，然后将这个Jacobian矩阵与grad_tensors参数对应的矩阵进行对应的点乘，得到最终的结果。